In [5]:
import sys, os, pickle, time, json, yaml, re
from glob import glob
from prettytable import PrettyTable

In [6]:
def make_results_table(version=None, cat=None, dirs=None, nPulls=4, skipDirs=None, verbose=False):
    if dirs is None and version is not None and cat is not None:
        results_dir = os.environ['HOME'] + '/public_html/BPH_RDst/Combine/'
        dirs = glob(results_dir + '*'+version+'*'+cat+'*')
        auxDirs = []
        for d in sorted(dirs, key=lambda path: os.stat(path).st_ctime):
            if (not os.path.basename(d).startswith('v'+version+'_')) and (not os.path.basename(d).startswith(version+'_')):
                continue
            if skipDirs is not None:
                if d in skipDirs:
                    continue
            auxDirs.append(d)
        dirs = auxDirs
    elif dirs is not None:
        auxDirs = []
        for d in sorted(dirs):
            if os.path.isdir(d):
                if d[-1] == '/':
                    d = d[:-1]
                auxDirs.append(d)
        dirs = auxDirs
    else:
        print 'Input not recognized'
        return
    
    if isinstance(dirs, list):
        if len(dirs) > 0:
            if verbose:
                print 'Fetching directories:'
                for d in dirs: print d
        else:
            print 'No existing directories'
            return
    else:
        print '[ERROR] dirs:', dirs
        return
    
    table = PrettyTable()
    table.field_names = ['Version', 'Sat. GoF', 'Scan [%]', 'Cat comp', 'Top pulls','[sig]']
    table.align['Top pulls'] = 'l'
    table.align['[sigma]'] = 'r'

    for dd in dirs:
        tag = os.path.basename(dd)
        if version is not None:
            tag = tag[tag.find(version+'_')+len(version+'_'):]
        if cat is not None:
            tag = tag[:tag.find(cat)]
        if tag.endswith('_'):
            tag = tag[:-1]
        if not tag:
            tag = '-'
        if len(tag) > 30:
            tag = tag[:30] + '...'
        # print 'Tag:', tag


        GoF_file = dd + '/GoF_results.txt'
        pval_sat = '-'
        if os.path.isfile(GoF_file):
            with open(GoF_file) as f:
                for line in reversed(f.readlines()):
                    if 'algoSat' in line:
                        break
                data = [x for x in line.split(' ') if x]
                data[2] = data[2][1:-2]
                pval_sat = data[1][:-4] + ' ('
                if float(data[2]) > 0:
                    pval_sat += data[2] + '%' + ')'
                else:
                    pval_sat += 'x{:.2f}'.format(float(data[1])/float(data[-2])) + ')'


        scan_file = dd + '/scan_results.txt'
        scan_result = '-'
        if os.path.isfile(scan_file):
            with open(scan_file) as f:
                line = f.readlines()[-1]
                data = [x for x in line.split(' ') if x]
                errUp = float(data[2][1:])
                errDw = float(data[4][1:])
                scan_result = ['{:.1f} + {:.1f}'.format(100*float(data[1]), 100*errUp)]
                scan_result.append(' '*scan_result[0].find('+') + '- {:.1f}'.format(100*errDw))
    #     print scan_result

        catComp_file = dd + '/categoriesCompatibility.txt'
        catComp = '-'
        if os.path.isfile(catComp_file):
            with open(catComp_file) as f:
                content = f.readlines()
                if len(content):
                    line = content[-1]
                    catComp = line.split(' ')[-1][1:-5] + '%'
    #     print catComp

        pulls_file = dd + '/scanNuisanceOut_Base.txt'
        pulls = [['-', '-']]*nPulls
        if os.path.isfile(pulls_file):
            with open(pulls_file) as f:
                for i, line in enumerate(f.readlines()[3:nPulls+3]):
                    pulls[i] = [x for x in line[:-1].replace('|', '').split(' ') if x]
                    if pulls[i][0].startswith('prop_bin'):
                        pulls[i][0] = '~'+pulls[i][0][len('prop_bin'):]

        if dd != dirs[0]:
            table.add_row(len(table.field_names)*[''])
        table.add_row([tag, pval_sat, scan_result[0], catComp, pulls[0][0], pulls[0][1]])
        for i in range(1, len(pulls)):
            table.add_row(['', '', scan_result[1] if i==1 else '', '', pulls[i][0], pulls[i][1]])

    return table

In [7]:
def print_with_title(table, title=''):
    if not title:
        print table
        return
    
    tw = len(table.get_string().split('\n')[0])
    
    if len(title) > tw-7:
        title = title[:tw-7] + '...'

    nb = tw - len(title) - 2
    nbb, nba = [nb/2, nb/2] if nb%2==0 else [(nb-1)/2, 1+(nb-1)/2]
    
    out = '+' + '-'*(tw-2) + '+' + '\n'
    out += '|' + nbb*' ' + title + nba*' ' + '|\n'
    out += table.get_string()
    out += '\n'
    print out

In [8]:
for ccc in ['low', 'mid', 'high', 'comb']:
    table = make_results_table(version='v15_1', cat=ccc)
    print_with_title(table, ccc)

+-----------------------------------------------------------------------------------------------------------+
|                                                    low                                                    |
+-----------------------------------+-------------+------------+----------+-------------------------+-------+
|              Version              |   Sat. GoF  |  Scan [%]  | Cat comp | Top pulls               | [sig] |
+-----------------------------------+-------------+------------+----------+-------------------------+-------+
|                 -                 | 229 (22.5%) | 0.0 + 12.9 |    -     | ~ctrl_mm_mHad_bin1      | -2.14 |
|                                   |             |     - 0.0  |          | brB_DstPiMuNu_1         |  1.95 |
|                                   |             |            |          | ctrlNormBToDstHcMu7_IP4 |  1.42 |
|                                   |             |            |          | ~ctrl_mm_mHad_bin11     | -1.19 |
|         

+------------------------------------------------------------------------------------------------------------+
|                                                    comb                                                    |
+-----------------------------------+-------------+-----------+----------+---------------------------+-------+
|              Version              |   Sat. GoF  |  Scan [%] | Cat comp | Top pulls                 | [sig] |
+-----------------------------------+-------------+-----------+----------+---------------------------+-------+
|    NOctrlNorm_NODXX_NOctrl2tks    | 485 (17.5%) | 0.0 + 5.5 |   100%   | B2DstCLNeig2              |  1.87 |
|                                   |             |     - 0.0 |          | Bd_DDs1Br                 |  1.83 |
|                                   |             |           |          | brB_DstPiMuNu_1           |  1.69 |
|                                   |             |           |          | ~low_ctrl_m__mHad_bin18   | -1.50 |
|

In [ ]:
ddd = [
    '/storage/af/user/alatorre/public_html/BPH_RDst/Combine/v14_1_bug_fix_control_regions_squashed3_dsthc_{}_CLN_blinded',
    '/storage/af/user/ocerri/public_html/BPH_RDst/Combine/v15_0_{}_CLN_blinded'
]

for ccc in ['low', 'mid', 'high', 'comb']:
    auxDir = [d.format(ccc) for d in ddd]
    print ccc
    print make_results_table(dirs=auxDir, cat=ccc)
    

In [ ]:
print make_results_table(version='14_1', cat='comb')

In [ ]:
# outfile = results_dir + 'summary_table_{}_{}.txt'.format(version, cat)
# os.system('date > '+outfile)
# with open(outfile, 'a') as f:
#     f.write(table.get_string())
    
# with open(outfile[:-4]+'.html', 'w') as f:
#     f.write(table.get_html_string())